In [6]:
from rex import WindX
from albatross.requests import get_regions, request_wtk_point_data, build_wtk_filepath, identify_regions
import pandas as pd

In [7]:
DATASETS = ['coordinates', 'inversemoninobukhovlength_2m', 'meta', 'precipitationrate_0m', 'pressure_0m', 'pressure_100m', 'pressure_200m', 'relativehumidity_2m', 'temperature_100m', 'temperature_10m', 'temperature_120m', 'temperature_140m', 'temperature_160m', 'temperature_200m', 'temperature_2m', 'temperature_40m', 'temperature_60m', 'temperature_80m', 'time_index', 'winddirection_100m', 'winddirection_10m', 'winddirection_120m', 'winddirection_140m', 'winddirection_160m', 'winddirection_200m', 'winddirection_40m', 'winddirection_60m', 'winddirection_80m', 'windspeed_100m', 'windspeed_10m', 'windspeed_120m', 'windspeed_140m', 'windspeed_160m', 'windspeed_200m', 'windspeed_40m', 'windspeed_60m', 'windspeed_80m']

In [8]:
get_regions(True)

{
    "bangladesh": {
        "year_range": [
            2014,
            2017
        ],
        "resolutions": [
            "hourly"
        ],
        "lat_lon_range": [
            [
                20.63102,
                28.500214
            ],
            [
                85.809204,
                93.72342
            ]
        ]
    },
    "canada": {
        "year_range": [
            2007,
            2014
        ],
        "resolutions": [
            "hourly",
            "5min"
        ],
        "lat_lon_range": [
            [
                40.0522,
                68.10168
            ],
            [
                -144.3318,
                -40.799896
            ]
        ]
    },
    "central_asia": {
        "year_range": [
            2015,
            2015
        ],
        "resolutions": [
            "hourly"
        ],
        "lat_lon_range": [
            [
                35.184048,
                55.425625
            ],
            [
      

{'bangladesh': {'year_range': [2014, 2017],
  'resolutions': ['hourly'],
  'lat_lon_range': [[20.63102, 28.500214], [85.809204, 93.72342]]},
 'canada': {'year_range': [2007, 2014],
  'resolutions': ['hourly', '5min'],
  'lat_lon_range': [[40.0522, 68.10168], [-144.3318, -40.799896]]},
 'central_asia': {'year_range': [2015, 2015],
  'resolutions': ['hourly'],
  'lat_lon_range': [[35.184048, 55.425625], [46.519226, 87.2984]]},
 'conus': {'year_range': [2007, 2014],
  'resolutions': ['hourly', '5min'],
  'lat_lon_range': [[23.833504, 49.3556], [-129.22923, -65.7146]]},
 'hawaii': {'year_range': [2000, 2019],
  'resolutions': ['hourly', '5min'],
  'base': 'Hawaii',
  'lat_lon_range': [[15.564247, 26.22171], [-164.45187, -151.27744]]},
 'india': {'year_range': [2014, 2014],
  'resolutions': ['hourly'],
  'lat_lon_range': [[4.2723618, 31.015549], [63.305073, 92.87215]]},
 'mexico': {'year_range': [2007, 2014],
  'resolutions': ['hourly', '5min'],
  'lat_lon_range': [[8.510117, 32.714092], [-

In [9]:
lat_lon = (39.913561, -105.222422)

In [10]:
identify_regions(lat_lon)

['conus']

In [11]:
identify_regions(lat_lon, coordinates=True)

[{'conus': array([[  37.603382, -127.61705 ],
         [  37.62042 , -127.62601 ],
         [  37.63745 , -127.63498 ],
         ...,
         [  40.369072,  -65.73694 ],
         [  40.386314,  -65.727936],
         [  40.403545,  -65.71893 ]], dtype=float32)}]

In [ ]:
year = 2010
params = ['windspeed_100m', 'winddirection_100m', 'pressure_100m']
(data, meta) = request_wtk_point_data(lat_lon, year, params)

In [ ]:
data